# Iteration

> As long as I'm alive, APL will never be used in Munich --_Fritz Bauer_<br>
> Nor in Holland --_Edsger Dijkstra (as told by Alan Perlis)_

We started all this by claiming that there are no loops in APL. This is of course not entirely true: there are plenty of ways of achieving iteration, some of which are more efficient than others. 

In order to get the best possible performance out of APL, it's worth seeking data-parallel algorithms, typically employing boolean masks. However, it's not always possible, or sometimes performance matters less than code complexity, and a more iterative solution can be both clearer and fast enough.

We have at least four, maybe five different kinds of iteration mechanisms at our disposal: map (`¨`), reduce (`⌿`), recur (`∇`) and power (`⍣`). The fifth is that of _scalar extension_, which can either be seen as a way of achieving iteration, or as a way of _avoiding_ iteration, depending on your point of view. Wait, is it six? Maybe we should count rank (`⍤`), too? Rank deserves its own separate section! Oh, and scan (`⍀`), don't forget scan! 

Let's introduce them.

In [1]:
⎕IO ← 0
]box on -s=min -t=tree -f=on
]rows on
assert←{⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}

done

Rebuilding user command cache... done

Was ON -style=min -trains=tree -fns=on

Was OFF

## Each (a.k.a map): `¨`

Most languages nowadays have a `map` construct. In fact, it's occasionally touted -- erroneously -- as sufficient evidence that a language is "functional" if it has a map function.

Perhaps you've seen Python's somewhat cumbersome version of map:

```python
>>> list(map(lambda x: x*x, [1, 2, 3, 4, 5, 6, 7, 8, 9])) # Square elements
[1, 4, 9, 16, 25, 36, 49, 64, 81]
```
which in Python corresponds to something like
```python
def mymap(func, iterable):
    for item in iterable:
        yield func(item)
```

Of course, seasoned Pythonistas would rightly frown at the above and instead recommend a _list comprehension_, but hey, who said anything about Python?

A map takes a function and applies it to every element in an array, creating a result array of the same length as its argument.

In APL, the glyph for map -- usually referred to as _each_ -- is two high dots: `¨`.

In [14]:
×⍨¨1+⍳9  ⍝ Square elements via each (but see below!)

┌→──────────────────────┐
│1 4 9 16 25 36 49 64 81│
└~──────────────────────┘

This wasn't actually a great example, this is one of those cases where _scalar extension_ already does the job for us:

In [13]:
×⍨1+⍳9  ⍝ Square elements via scalar extension

┌→──────────────────────┐
│1 4 9 16 25 36 49 64 81│
└~──────────────────────┘

Scalar extension means that functions in certain cases already know how to penetrate arrays.

Let's try another. Given a nested vector, what are the lengths of the elements?

In [10]:
⊢V ← (1 2 3 4)(1 2)(3 4 5 6 7)(,2)(5 4 3 2 1)
≢¨V ⍝ Tally-each

┌→────────────────────────────────────────────┐
│ ┌→──────┐ ┌→──┐ ┌→────────┐ ┌→┐ ┌→────────┐ │
│ │1 2 3 4│ │1 2│ │3 4 5 6 7│ │2│ │5 4 3 2 1│ │
│ └~──────┘ └~──┘ └~────────┘ └~┘ └~────────┘ │
└∊────────────────────────────────────────────┘

┌→────────┐
│4 2 5 1 5│
└~────────┘

If you're used to a different language, "each" has a seductive quality, as it maps conceptually onto constructs you already know how to use. Beware though that in APL it's often inefficient, and that there are usually better alternatives.

## Reduce (a.k.a foldr): `⌿`, `/`

Most languages today sport some kind of variety of _fold_ or _reduce_, regardless of the level of "functional" they claim to be.

In APL, it's a central feature, which somewhat unhelpfully hijacks the glyph also used by compress/replicate, `/`.  This is one, albeit not the main, resaon why we'll use its close cousin, _reduce-first_, `⌿`, where we can. Reduction has a bit of an unfair reputation of being hard to understand. Guido reportedly hates "reduce" in Python so much that it was demoted down to the dusty "functools" module.

In fact, reduction is a pretty simple idea, and APL may even have been the first programming language [with reduce in it](https://www.jsoftware.com/papers/APL1.htm#1.8) (some Lispers disagree). Think of the operation of summing a bunch of numbers -- this is an example of a reduce.

In APL, reduce applies a function to elements in an array, producing a result which is rank-reduced by 1. In other words, reducing a vector (rank 1) produces a scalar (rank 0). In the example of _sum_, summing the elements of a vector obviously produces a scalar: the total.

In [21]:
+⌿1 2 3 4 5 6 7 8 9 ⍝ sum-reduce integers 1-9

45

We can, somewhat simplified, think of the reduce-first operator injecting its left operand function in the gaps between elements in the argument:

In [29]:
1+2+3+4+5+6+7+8+9

45

When using reduce in APL, you need to take extra care to ensure that it works with its strict right to left evaluation order. A reduce is also called a _fold_ in other languages (Lisp, Erlang etc), and APL's reduce is a so-called _foldr_ -- it reduces right to left, which makes sense for APL, but occasionally less sense for the programmer.

Again, it can help writing it out in long-hand to see what's going on:

In [23]:
-⌿1 2 3 4 5 6 7 8 9 ⍝ difference-reduction -- take care: right to left fold!

5

If that was the result you expected, you're well on your way to mastery. Inject the operand between items:

In [28]:
1-2-3-4-5-6-7-8-9

5

Reduction is especially useful when working with higher-rank arrays. Reduce-first is so called because it reduces along the _first_ axis. So a sum-reduce-first of a rank-2 integer array will sum its columns to produce a _vector_ (rank 1) of the columnar sums:

In [31]:
⊢m ← 3 3⍴9?9
+⌿m

┌→────┐
↓1 2 3│
│4 8 0│
│6 5 7│
└~────┘

┌→───────┐
│11 15 10│
└~───────┘

If we wanted to sum-reduce along the rows, we can either use `/` which for historical reasons does just that:

In [32]:
+/m

┌→──────┐
│6 12 18│
└~──────┘

or we can explicitly tell `⌿` to apply along a different axis:

In [33]:
+⌿[1]m

┌→──────┐
│6 12 18│
└~──────┘

In general, for consistency it's best to prefer operators and functions that default to applying to the leading axis where possible. The fact that APL, unlike J, has a mixture is somewhat of an unhelpful side-effect of backwards compatibility.

## Windowed reduction

Reduce has a few more handy tricks up its sleeve. Instead of reducing the whole argument array, we can employ a sliding window and so instead compute a set of reductions over shorter stretches of the data. The derived function returned by the reduction operators can be called dyadically, specifying as the left argument the size of the sliding window. 

For example, to calculate the sum of each element in a vector with its subsequent element, we employ a sliding reduction window of size 2:

In [6]:
2+⌿1 2 3 4 5 6 7 8 9

┌→──────────────────┐
│3 5 7 9 11 13 15 17│
└~──────────────────┘

## Scan: `⍀, \`

Scan blurrs the distinction between map and reduce. In right hands, it can be a true APL super power, but beware that scans tend to be slow. 

Scan is just like reduce, but instead returns every intermediate state, not just the end state. A sum-reduce of a vector of numbers returns the sum total. A sum-scan of the same vector returns the running sums:

In [3]:
+⌿1 2 3 4 5 6 7 8 9
+⍀1 2 3 4 5 6 7 8 9

45

┌→──────────────────────┐
│1 3 6 10 15 21 28 36 45│
└~──────────────────────┘

One way we can think of scan is that it's the amalgamation of all possible calls to reduce with the same operand, taking in increasing lengths of the argument array. In the case above:

In [4]:
+⌿1
+⌿1 2
+⌿1 2 3
+⌿1 2 3 4
+⌿1 2 3 4 5
+⌿1 2 3 4 5 6
+⌿1 2 3 4 5 6 7 
+⌿1 2 3 4 5 6 7 8
+⌿1 2 3 4 5 6 7 8 9

1

3

6

10

15

21

28

36

45

## Power: `⍣`

One of my favourite glyphs! It looks like a happy starfish! The power operator is... powerful. Conceptually it should be easy to grasp, but there are some aspects that take time to understand. Formally, it's defined as a function repeatedly applied to the output of itself, until some stopping criterion is fulfilled. If you pass it an integer as its right operand, it's basically a for-loop:

```apl
f ← { ... }               ⍝ some function or other
f f f f f f f f argvector ⍝ repeatedly apply function to itself, eh 8 times
f⍣8 ⊢ argvector           ⍝ power-8
```

If you give it a function as the right operand, it can be used as a while-loop. One example is to find a function's _fixpoint_:

In [7]:
2÷⍨⍣=10 ⍝ Divide by 2 until fixpoint

0

Here the right operand function is _equals_ `=`. This says: repeatedly apply the left operand (`2÷⍨`) until two subsequent applications return the same value.

We can explicitly refer to the left and right arguments of the right operand function. The left argument, `⍺`, refers to the result of the function application to the right argument, `⍵`. 

Keep generating random numbers between 1 and 10 until we get a 6:

In [2]:
 {⍞ ← ?10}⍣{6=⍺} 0 ⍝ Keep generating random numbers between 1 and 10 until we get a 6

43410153210102598104226
6

The quad-quote-gets (`⍞←`) combo prints values without newlines. The final result is also returned, the expected 6.

## Recur: `∇`

Dyalog has a most excellent, concise and efficient recursion operator. It allows you to express recursive algorithms in a natural, almost lisp-like fashion. The interpreter has a very good [TCO](https://en.wikipedia.org/wiki/Tail_call) implementation.

Let's start with making our own version of sum-reduce, this time without actually using the reduce operator.

In [34]:
]dinput
Sum ← {
    ⍺ ← 0        ⍝ Left arg defaults to 0 if not given
    0=≢⍵:⍺       ⍝ If right arg is empty, return left arg
    (⍺+⊃⍵)∇1↓⍵   ⍝ Add head to acc, recur over tail
}

In [35]:
⊢mysum ← Sum 1 2 3 4 5 6 7 8 9
assert mysum=+/1 2 3 4 5 6 7 8 9

45

Yup, that seems to work, good.

The glyph recur, `∇` is a reference to the current innermost dfn. If your dfn has a name, you can substitute it for the actual function name. In our case, the last line could equally well have been written:
```apl
(⍺+⊃⍵)sum 1↓⍵
```
However, using the glyph has a number of advantages: it's more concise, immune to function name changes, and works equally well for anonymous dfns.

Our `sum` dfn follows a common pattern. We accumulate something as the left argument, and decrease the right argument, either by magnitude, or as in this case, by dropping items off the front of a vector.

The recursion termination clause,
```apl
0=≢⍵:⍺
```
simply states that if the right argument is empty, we should return our accumulator. The recusive call itself is "add the head of the right argument to the accumulator, and recur on this to the left, and the tail to the right".

If the last thing the function does is a function call, and this includes a _recur_ call, this is what's called a _tail call_ which the Dyalog interpreter can handle without the addition of an extra stack frame. If you're using recur, strive to make your functions tail-recursive -- avoid making your function do any work on the value you get back from the recurring line.

For a fractionally more involved example, let's write our own _sum-scan_.

In [36]:
]dinput
Sscan ← {
    ⍺ ← ⍬              ⍝ Left arg defaults to ⍬ if not given
    0=≢⍵:⍺             ⍝ If right arg is empty, return left arg
    (⍺,⊃⍵+⊃¯1↑⍺)∇1↓⍵   ⍝ Append the sum of the head and the last element of acc and recur on tail
}

In [37]:
⊢myscan ← Sscan 1 2 3 4 5 6 7 8 9
assert myscan≡+⍀1 2 3 4 5 6 7 8 9

┌→──────────────────────┐
│1 3 6 10 15 21 28 36 45│
└~──────────────────────┘

No discourse on recursion is complete without mentioning the _Fibonacci_ sequence. You know which one I mean -- starting at 0 1, every subsequent number is the sum of its two direct predecessors:

    0 1 1 2 3 5 8 13 21 34 ⍝ etc
    
Here's one possible formulation where the right argument is the Fibonacci ordinal.

In [38]:
]dinput
Fib←{ ⍝ Tail-recursive Fibonacci.
    ⍺ ← 0 1
    ⍵=0:⊃⍺
    (1↓⍺,+/⍺)∇⍵-1
}

In [39]:
Fib¨⍳10 ⍝ The 10 first Fibonacci numbers

┌→─────────────────────┐
│0 1 1 2 3 5 8 13 21 34│
└~─────────────────────┘

The pattern is still the same: set a default for the accumulator, `⍺`. Terminate on some condition on `⍵`, returning a function of the accumulator. Modify the accumulator, given the head of the right argument, and recur on the tail.

The guts of the function is the last line. To the right, we decrease the right argument -- this is our loop counter if you like. To the left is our accumulator, which basically is a sliding window of size 2 over the Fib sequence. We append the sum of the two numbers, and drop the first, and recur over the tail.

Here's a pretty neat implementation of the [quicksort](https://en.wikipedia.org/wiki/Quicksort) algorithm:

In [2]:
]dinput
Quicksort←{
    1≥≢⍵:⍵
    S←{⍺⌿⍨⍺ ⍺⍺ ⍵}
    ⍵((∇<S),=S,(∇>S))⍵⌷⍨?≢⍵
}

Here `⍵⌷⍨?≢⍵` is the pivot element, picked at random, and the `S` operator partitions its left argument array based on its left operand function and the pivot element to the right. The whole idea of quicksort is pretty clearly visible in the [tacit](tacit.ipynb) fork `(∇<S),=S,(∇>S)` -- elements _less_ that pivot, pivot, elements _greater than_ pivot, recursively applied.

In [4]:
Quicksort ⎕←20?20

15 14 8 5 10 7 20 18 1 13 16 2 17 19 11 9 3 12 4 6
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20

Another example is binary search: locate an element in an array that is known to be sorted. Here's a function to do that:

In [2]:
]dinput
bsearch←{⎕IO←0
    0 (⍺ {                  ⍝ Operator: ⍺,⍵ - lower,upper index. ⍺⍺ - item, ⍵⍵ - array
        ⍺>⍵:¯1              ⍝ If lower index has moved past upper, item's not present
        mid ← ⌈0.5×⍺+⍵      ⍝ New midpoint  
        ⍺⍺=mid⊃⍵⍵:mid       ⍝ Check if item is at the new midpoint
        ⍺⍺<mid⊃⍵⍵:⍺∇¯1+mid  ⍝ Drill into lower half
        (1+mid)∇⍵           ⍝ Upper half
    }(,⍵)) ¯1+≢,⍵
}

In [3]:
5 bsearch 0 2 3 5 8 12 75
5 bsearch 0 2 3 5 8 12
5 bsearch 5 5
5 bsearch 5
1 bsearch 0 2 3 5 8 12

3

3

1

0

¯1

## Scalar extension

We touched briefly on _scalar extension_ above, but it's an important topic, so let's dive in a little bit deeper. It's worth reading what Dyalog has to say on the topic in the [docs](http://help.dyalog.com/18.0/index.htm#Language/Primitive%20Functions/Scalar%20Functions.htm).

The idea is that a certain class of functions is _pervasive_. With this we mean that a function that operates on a scalar will operate on scalars at any level of nesting if applied to an array. Note that the level of nesting is _not_ the same as the rank in APL (rank and depth _are_ equivalent in k though). 

Consider a nested vector that contains both numbers and other vectors of numbers:

In [41]:
⊢nesty ← (1 2 3 (3 4 (5 6)) 7)

┌→──────────────────────┐
│ ┌→──────────┐ │
│ 1 2 3 │ ┌→──┐ │ 7 │
│ │ 3 4 │5 6│ │ │
│ │ └~──┘ │ │
│ └∊──────────┘ │
└∊──────────────────────┘

As an obvious and simple example, let's say we want to negate all the numbers:

In [42]:
-nesty

┌→──────────────────────────────┐
│ ┌→──────────────┐ │
│ ¯1 ¯2 ¯3 │ ┌→────┐ │ ¯7 │
│ │ ¯3 ¯4 │¯5 ¯6│ │ │
│ │ └~────┘ │ │
│ └∊──────────────┘ │
└∊──────────────────────────────┘

Where it can be applied, scalar extension is an efficient operation in Dyalog APL. It works for dyads, too:

In [44]:
(1 2) 3 + 4 (5 6)
(1 1⍴5) - 1 (2 3)

┌→────────────┐
│ ┌→──┐ ┌→──┐ │
│ │5 6│ │8 9│ │
│ └~──┘ └~──┘ │
└∊────────────┘

┌→────────┐
│ ┌→──┐ │
│ 4 │3 2│ │
│ └~──┘ │
└∊────────┘